THIS PROJECT IS COMPRISED OF THREE FILES; ONE FOR DATA PREPARATION - DATA MUNGING. THE OTHER TWO FILES ARE FOR FITTING
MODELS TO THE PREPARED DATA AND EVALUATING THIER PERFORMANCE BASED ON DIFFERENNT METRICS AND THE STATUS OF THE DATA.

THE AIM OF THE PROJECT WILL TO PREDICT USING MACHINE LEARNING CLASSIFIERS WHETHER A PATIENT HAS TYPE B DIABETES OR NOT [THE MODELS WILL CLASSIFY THE PATIENTS AS SUCH, BASED ON THEIR MEDICAL DATA]

THE SELECT CLASSIFIERS ARE RANDOM FORESTS (RF), SUPPORT VECTOR MACHINE(SVM), AND EXTREME GRADIENT BOOSTERS (XGB). THIS MODELS ARE FAST TO TRAIN, SCORE GENERALLY WELL, HAVE A STRONG UNDERLYING ARCHITECTURE & LOGIC FOR CLASSIFICATION, AND HAVE A FAIR PERFORMANCE INDEX ON BOTH SMALL AND HUGE DATASETS COMPARED TO OTHER CLASSIFIERS, HENCE BEST SUITED FOR THIS PROJECT. 

In [49]:
#importing support libraries for data preparation
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib
import seaborn as sns
import plotly.express as px


In [50]:
#setting default parameters for visualization
sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#000000'

In [51]:
#import the dataset
df_raw = pd.read_csv('../datasets/diabetes.csv')

EXPLORATORY DATA ANALYSIS

INCLUDES DATA SHAPE, DATA TYPES, AND CHECK FOR MISSSING VALUES

In [52]:
df_raw

,id,chol,stab.glu,hdl,ratio,glyhb,location,age,gender,height,weight,frame,bp.1s,bp.1d,bp.2s,bp.2d,waist,hip,time.ppn
0,1000,203.0,82,56.0,3.6,4.310000,Buckingham,46,female,62.0,121.0,medium,118.0,59.0,NaN,NaN,29.0,38.0,720.0
1,1001,165.0,97,24.0,6.9,4.440000,Buckingham,29,female,64.0,218.0,large,112.0,68.0,NaN,NaN,46.0,48.0,360.0
2,1002,228.0,92,37.0,6.2,4.640000,Buckingham,58,female,61.0,256.0,large,190.0,92.0,185.0,92.0,49.0,57.0,180.0
3,1003,78.0,93,12.0,6.5,4.630000,Buckingham,67,male,67.0,119.0,large,110.0,50.0,NaN,NaN,33.0,38.0,480.0
4,1005,249.0,90,28.0,8.9,7.720000,Buckingham,64,male,68.0,183.0,medium,138.0,80.0,NaN,NaN,44.0,41.0,300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,41506,296.0,369,46.0,6.4,16.110001,Louisa,53,male,69.0,173.0,medium,138.0,94.0,130.0,94.0,35.0,39.0,210.0
399,41507,284.0,89,54.0,5.3,4.390000,Louisa,51,female,63.0,154.0,medium,140.0,100.0,146.0,102.0,32.0,43.0,180.0
400,41510,194.0,269,38.0,5.1,13.630000,Louisa,29,female,69.0,167.0,small,120.0,70.0,NaN,NaN,33.0,40.0,20.0
401,41752,199.0,76,52.0,3.8,4.490000,Louisa,41,female,63.0,197.0,medium,120.0,78.0,NaN,NaN,41.0,48.0,255.0


In [53]:
df_raw.shape

(403, 19)

In [54]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        403 non-null    int64  
 1   chol      402 non-null    float64
 2   stab.glu  403 non-null    int64  
 3   hdl       402 non-null    float64
 4   ratio     402 non-null    float64
 5   glyhb     390 non-null    float64
 6   location  403 non-null    object 
 7   age       403 non-null    int64  
 8   gender    403 non-null    object 
 9   height    398 non-null    float64
 10  weight    402 non-null    float64
 11  frame     391 non-null    object 
 12  bp.1s     398 non-null    float64
 13  bp.1d     398 non-null    float64
 14  bp.2s     141 non-null    float64
 15  bp.2d     141 non-null    float64
 16  waist     401 non-null    float64
 17  hip       401 non-null    float64
 18  time.ppn  400 non-null    float64
dtypes: float64(13), int64(3), object(3)
memory usage: 59.9+ KB


In [55]:
df_raw.isnull().sum() 

id            0
chol          1
stab.glu      0
hdl           1
ratio         1
glyhb        13
location      0
age           0
gender        0
height        5
weight        1
frame        12
bp.1s         5
bp.1d         5
bp.2s       262
bp.2d       262
waist         2
hip           2
time.ppn      3
dtype: int64

FEATURE ENGINEERING 

EXTRACTING NEW FEATURES FROM EXISTING FEATURES

In [56]:
#create a new column for height in meters by converting current height from inches
df_raw['height_meters'] = df_raw['height'] * 0.0254

In [57]:
#BMI can be a good determinant in diagnosing heart disease and diabetes
#calculating and adding a column for the BMI using height and weight
df_raw['BMI'] = df_raw['weight'] / (df_raw['height_meters'])**2

In [58]:
df_raw

,id,chol,stab.glu,hdl,ratio,glyhb,location,age,gender,height,...,frame,bp.1s,bp.1d,bp.2s,bp.2d,waist,hip,time.ppn,height_meters,BMI
0,1000,203.0,82,56.0,3.6,4.310000,Buckingham,46,female,62.0,...,medium,118.0,59.0,NaN,NaN,29.0,38.0,720.0,1.5748,48.790420
1,1001,165.0,97,24.0,6.9,4.440000,Buckingham,29,female,64.0,...,large,112.0,68.0,NaN,NaN,46.0,48.0,360.0,1.6256,82.495282
2,1002,228.0,92,37.0,6.2,4.640000,Buckingham,58,female,61.0,...,large,190.0,92.0,185.0,92.0,49.0,57.0,180.0,1.5494,106.638214
3,1003,78.0,93,12.0,6.5,4.630000,Buckingham,67,male,67.0,...,large,110.0,50.0,NaN,NaN,33.0,38.0,480.0,1.7018,41.089412
4,1005,249.0,90,28.0,8.9,7.720000,Buckingham,64,male,68.0,...,medium,138.0,80.0,NaN,NaN,44.0,41.0,300.0,1.7272,61.343116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,41506,296.0,369,46.0,6.4,16.110001,Louisa,53,male,69.0,...,medium,138.0,94.0,130.0,94.0,35.0,39.0,210.0,1.7526,56.322314
399,41507,284.0,89,54.0,5.3,4.390000,Louisa,51,female,63.0,...,medium,140.0,100.0,146.0,102.0,32.0,43.0,180.0,1.6002,60.141214
400,41510,194.0,269,38.0,5.1,13.630000,Louisa,29,female,69.0,...,small,120.0,70.0,NaN,NaN,33.0,40.0,20.0,1.7526,54.368939
401,41752,199.0,76,52.0,3.8,4.490000,Louisa,41,female,63.0,...,medium,120.0,78.0,NaN,NaN,41.0,48.0,255.0,1.6002,76.933890


In [59]:
df_raw.columns

Index(['id', 'chol', 'stab.glu', 'hdl', 'ratio', 'glyhb', 'location', 'age',
       'gender', 'height', 'weight', 'frame', 'bp.1s', 'bp.1d', 'bp.2s',
       'bp.2d', 'waist', 'hip', 'time.ppn', 'height_meters', 'BMI'],
      dtype='object')

In [60]:
#Hip-waist ratio is a good determinant to heart disease
#create a column for this and add it to the dataset
df_raw['hip_waist_ratio'] = df_raw['waist'] / df_raw['hip']

In [61]:
#A patient is diagnosed positive for diabetes if their 'glyhb' is above 7
#extracting a feature from this, will give us the target feature for our supervised ML models
def condition(df_raw):
    if (df_raw['glyhb'] < 7):
        return 0
    else:
        return 1

df_raw['diabetic'] = df_raw.apply(condition, axis=1)

In [62]:
df_raw

,id,chol,stab.glu,hdl,ratio,glyhb,location,age,gender,height,...,bp.1d,bp.2s,bp.2d,waist,hip,time.ppn,height_meters,BMI,hip_waist_ratio,diabetic
0,1000,203.0,82,56.0,3.6,4.310000,Buckingham,46,female,62.0,...,59.0,NaN,NaN,29.0,38.0,720.0,1.5748,48.790420,0.763158,0
1,1001,165.0,97,24.0,6.9,4.440000,Buckingham,29,female,64.0,...,68.0,NaN,NaN,46.0,48.0,360.0,1.6256,82.495282,0.958333,0
2,1002,228.0,92,37.0,6.2,4.640000,Buckingham,58,female,61.0,...,92.0,185.0,92.0,49.0,57.0,180.0,1.5494,106.638214,0.859649,0
3,1003,78.0,93,12.0,6.5,4.630000,Buckingham,67,male,67.0,...,50.0,NaN,NaN,33.0,38.0,480.0,1.7018,41.089412,0.868421,0
4,1005,249.0,90,28.0,8.9,7.720000,Buckingham,64,male,68.0,...,80.0,NaN,NaN,44.0,41.0,300.0,1.7272,61.343116,1.073171,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,41506,296.0,369,46.0,6.4,16.110001,Louisa,53,male,69.0,...,94.0,130.0,94.0,35.0,39.0,210.0,1.7526,56.322314,0.897436,1
399,41507,284.0,89,54.0,5.3,4.390000,Louisa,51,female,63.0,...,100.0,146.0,102.0,32.0,43.0,180.0,1.6002,60.141214,0.744186,0
400,41510,194.0,269,38.0,5.1,13.630000,Louisa,29,female,69.0,...,70.0,NaN,NaN,33.0,40.0,20.0,1.7526,54.368939,0.825000,1
401,41752,199.0,76,52.0,3.8,4.490000,Louisa,41,female,63.0,...,78.0,NaN,NaN,41.0,48.0,255.0,1.6002,76.933890,0.854167,0


In [63]:
#assessing the distribution of the target variable 'diabetic'
df_raw.groupby('diabetic').count()


,id,chol,stab.glu,hdl,ratio,glyhb,location,age,gender,height,...,bp.1s,bp.1d,bp.2s,bp.2d,waist,hip,time.ppn,height_meters,BMI,hip_waist_ratio
diabetic,,,,,,,,,,,,,,,,,,,,,
0,330,329,330,329,329,330,330,330,330,326,...,325,325,107,107,328,328,327,326,326,328
1,73,73,73,73,73,60,73,73,73,72,...,73,73,34,34,73,73,73,72,71,73


In [64]:
#preserving a copy of the data before encoding it - this is ease future changes.
df_original= df_raw.copy()

In [65]:
#using label encoder to fit and transform all non numeric values into numerical values
for column in df_raw.columns:
    if df_raw[column].dtype == np.number:
        continue
    df_raw[column] = LabelEncoder().fit_transform(df_raw[column])

<ipython-input-65-8c5c99acdb75>:2: DeprecationWarning:

Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.



In [66]:
#preserving a copy of the now encoded data before proceeding
df_encoded = df_raw.copy()

In [67]:
df_raw

,id,chol,stab.glu,hdl,ratio,glyhb,location,age,gender,height,...,bp.1d,bp.2s,bp.2d,waist,hip,time.ppn,height_meters,BMI,hip_waist_ratio,diabetic
0,0,203.0,27,56.0,3.6,4.310000,0,27,0,62.0,...,59.0,NaN,NaN,29.0,38.0,720.0,1.5748,48.790420,0.763158,0
1,1,165.0,42,24.0,6.9,4.440000,0,10,0,64.0,...,68.0,NaN,NaN,46.0,48.0,360.0,1.6256,82.495282,0.958333,0
2,2,228.0,37,37.0,6.2,4.640000,0,39,0,61.0,...,92.0,185.0,92.0,49.0,57.0,180.0,1.5494,106.638214,0.859649,0
3,3,78.0,38,12.0,6.5,4.630000,0,48,1,67.0,...,50.0,NaN,NaN,33.0,38.0,480.0,1.7018,41.089412,0.868421,0
4,4,249.0,35,28.0,8.9,7.720000,0,45,1,68.0,...,80.0,NaN,NaN,44.0,41.0,300.0,1.7272,61.343116,1.073171,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,398,296.0,113,46.0,6.4,16.110001,1,34,1,69.0,...,94.0,130.0,94.0,35.0,39.0,210.0,1.7526,56.322314,0.897436,1
399,399,284.0,34,54.0,5.3,4.390000,1,32,0,63.0,...,100.0,146.0,102.0,32.0,43.0,180.0,1.6002,60.141214,0.744186,0
400,400,194.0,105,38.0,5.1,13.630000,1,10,0,69.0,...,70.0,NaN,NaN,33.0,40.0,20.0,1.7526,54.368939,0.825000,1
401,401,199.0,21,52.0,3.8,4.490000,1,22,0,63.0,...,78.0,NaN,NaN,41.0,48.0,255.0,1.6002,76.933890,0.854167,0


In [68]:
#finally after having all numeric values, we embark on imputation of missing values
#Machine Learning models cannot evaluate NaN hence this a key step in data preparation
df_raw.isnull().sum()

id                   0
chol                 1
stab.glu             0
hdl                  1
ratio                1
glyhb               13
location             0
age                  0
gender               0
height               5
weight               1
frame                0
bp.1s                5
bp.1d                5
bp.2s              262
bp.2d              262
waist                2
hip                  2
time.ppn             3
height_meters        5
BMI                  6
hip_waist_ratio      2
diabetic             0
dtype: int64

In [69]:
#First we do visualizations of different features to assess their distributions before and after we impute the missing values
#This is important as it will help in choosing the best features to fit the models.

#Here is a visualization of bp.2s - which has 262 missing values before imputation. 
fig = px.histogram(df_raw, x='bp.2s', marginal='box', nbins=47, title='bp.2s Distribution')
fig.update_layout(bargap=0.1)
fig.show()

In [70]:
fig = px.histogram(df_raw, x='bp.2d', marginal='box', nbins=47, title='bp.2d Distribution')
fig.update_layout(bargap=0.1)
fig.show()

In [71]:
df_dropped = df_raw.copy()

In [72]:
#we drop (delete) columns that do not show significant distributions and ones that are repeated
df_dropped.drop(columns='height')
df_dropped.drop(columns='id')
df_dropped.columns

Index(['id', 'chol', 'stab.glu', 'hdl', 'ratio', 'glyhb', 'location', 'age',
       'gender', 'height', 'weight', 'frame', 'bp.1s', 'bp.1d', 'bp.2s',
       'bp.2d', 'waist', 'hip', 'time.ppn', 'height_meters', 'BMI',
       'hip_waist_ratio', 'diabetic'],
      dtype='object')

In [73]:
df_dropped

,id,chol,stab.glu,hdl,ratio,glyhb,location,age,gender,height,...,bp.1d,bp.2s,bp.2d,waist,hip,time.ppn,height_meters,BMI,hip_waist_ratio,diabetic
0,0,203.0,27,56.0,3.6,4.310000,0,27,0,62.0,...,59.0,NaN,NaN,29.0,38.0,720.0,1.5748,48.790420,0.763158,0
1,1,165.0,42,24.0,6.9,4.440000,0,10,0,64.0,...,68.0,NaN,NaN,46.0,48.0,360.0,1.6256,82.495282,0.958333,0
2,2,228.0,37,37.0,6.2,4.640000,0,39,0,61.0,...,92.0,185.0,92.0,49.0,57.0,180.0,1.5494,106.638214,0.859649,0
3,3,78.0,38,12.0,6.5,4.630000,0,48,1,67.0,...,50.0,NaN,NaN,33.0,38.0,480.0,1.7018,41.089412,0.868421,0
4,4,249.0,35,28.0,8.9,7.720000,0,45,1,68.0,...,80.0,NaN,NaN,44.0,41.0,300.0,1.7272,61.343116,1.073171,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,398,296.0,113,46.0,6.4,16.110001,1,34,1,69.0,...,94.0,130.0,94.0,35.0,39.0,210.0,1.7526,56.322314,0.897436,1
399,399,284.0,34,54.0,5.3,4.390000,1,32,0,63.0,...,100.0,146.0,102.0,32.0,43.0,180.0,1.6002,60.141214,0.744186,0
400,400,194.0,105,38.0,5.1,13.630000,1,10,0,69.0,...,70.0,NaN,NaN,33.0,40.0,20.0,1.7526,54.368939,0.825000,1
401,401,199.0,21,52.0,3.8,4.490000,1,22,0,63.0,...,78.0,NaN,NaN,41.0,48.0,255.0,1.6002,76.933890,0.854167,0


In [74]:
#safe the dataset after deleting unwanted columns
df_dropped.to_csv('diabetes_missing_un_dropped_unfilled.csv', index=None)

In [75]:
#filling up missing values using the mean. 
df_raw.fillna(df_raw.mean(), inplace=True)

In [76]:
#save a copy of the dataset
df_raw.to_csv('diabetes_undropped_filled.csv', index=None)

In [77]:
#checking for missing values in the updated dataset - indication - None!
df_raw.isnull().sum()

id                 0
chol               0
stab.glu           0
hdl                0
ratio              0
glyhb              0
location           0
age                0
gender             0
height             0
weight             0
frame              0
bp.1s              0
bp.1d              0
bp.2s              0
bp.2d              0
waist              0
hip                0
time.ppn           0
height_meters      0
BMI                0
hip_waist_ratio    0
diabetic           0
dtype: int64

In [78]:
df_raw

,id,chol,stab.glu,hdl,ratio,glyhb,location,age,gender,height,...,bp.1d,bp.2s,bp.2d,waist,hip,time.ppn,height_meters,BMI,hip_waist_ratio,diabetic
0,0,203.0,27,56.0,3.6,4.310000,0,27,0,62.0,...,59.0,152.382979,92.524823,29.0,38.0,720.0,1.5748,48.790420,0.763158,0
1,1,165.0,42,24.0,6.9,4.440000,0,10,0,64.0,...,68.0,152.382979,92.524823,46.0,48.0,360.0,1.6256,82.495282,0.958333,0
2,2,228.0,37,37.0,6.2,4.640000,0,39,0,61.0,...,92.0,185.000000,92.000000,49.0,57.0,180.0,1.5494,106.638214,0.859649,0
3,3,78.0,38,12.0,6.5,4.630000,0,48,1,67.0,...,50.0,152.382979,92.524823,33.0,38.0,480.0,1.7018,41.089412,0.868421,0
4,4,249.0,35,28.0,8.9,7.720000,0,45,1,68.0,...,80.0,152.382979,92.524823,44.0,41.0,300.0,1.7272,61.343116,1.073171,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,398,296.0,113,46.0,6.4,16.110001,1,34,1,69.0,...,94.0,130.000000,94.000000,35.0,39.0,210.0,1.7526,56.322314,0.897436,1
399,399,284.0,34,54.0,5.3,4.390000,1,32,0,63.0,...,100.0,146.000000,102.000000,32.0,43.0,180.0,1.6002,60.141214,0.744186,0
400,400,194.0,105,38.0,5.1,13.630000,1,10,0,69.0,...,70.0,152.382979,92.524823,33.0,40.0,20.0,1.7526,54.368939,0.825000,1
401,401,199.0,21,52.0,3.8,4.490000,1,22,0,63.0,...,78.0,152.382979,92.524823,41.0,48.0,255.0,1.6002,76.933890,0.854167,0


DATA VISUALIZATION

THIS STEP HELPS IN THE FEATURE SELECTION PROCESS. FEATURES THAT HAVE GOOD STATISTICAL DISTRIBUTIONS WILL BE MOST PREFERREF
OVER ONES THAT DO NOT TAKE ANY PARTICULAR DISTRIBUTION OR TREND

In [79]:
#visualization of 'bp.2d' after imputing the 262 missing data points. The feature takes no particular distribution as
# a majority of the values are centered at the mean
fig = px.histogram(df_raw, x='bp.2d', marginal='box', nbins=47, title='bp.2d Distribution')
fig.update_layout(bargap=0.1)
fig.show()

In [80]:
#similar case for the 'bp.2s' feature - most values are centered on the mean
#this two features are a possible source of bias hence they will eliminated on feature selection in future steps
fig = px.histogram(df_raw, x='bp.2s', marginal='box', nbins=47, title='bp.2d Distribution')
fig.update_layout(bargap=0.1)
fig.show()

In [81]:
#the id takes no distribution hence not a suitable feature 
fig = px.histogram(df_raw, x='id', marginal='box', nbins=47, title='ID Distribution')
fig.update_layout(bargap=0.1)
fig.show()

In [82]:
#a normal distribution for the frame - body type
fig = px.histogram(df_raw, x='frame', marginal='box', nbins=4, title='frame Distribution')
fig.update_layout(bargap=0.1)
fig.show()

In [83]:
fig = px.histogram(df_raw, x='ratio', marginal='box', nbins=47, title='ratio Distribution')
fig.update_layout(bargap=0.1)
fig.show()

In [84]:
#few visible outliers on this feature - checking for correlation will help determine its suitability for modelling
fig = px.histogram(df_raw, x='time.ppn', marginal='box', nbins=47, title='time.ppn Distribution')
fig.update_layout(bargap=0.1)
fig.show()

In [85]:
fig = px.histogram(df_raw, x='hip_waist_ratio', marginal='box', nbins=47, title='hip_waist_ratio Distribution')
fig.update_layout(bargap=0.1)
fig.show()

In [86]:
#location may not be a significant feature there is an equal number of patients from the two locations used in the study
fig = px.histogram(df_raw, x='location', marginal='box', nbins=2, title='location Distribution')
fig.update_layout(bargap=0.1)
fig.show()

In [87]:
fig = px.histogram(df_raw, x='age', marginal='box', nbins=47, title='Age Distribution')
fig.update_layout(bargap=0.1)
fig.show()

In [88]:
fig = px.histogram(df_raw, x='hdl', marginal='box', nbins=47, title='hdl Distribution')
fig.update_layout(bargap=0.1)
fig.show()

In [89]:
fig = px.histogram(df_raw, x='gender', marginal='box', nbins=2, title='gender Distribution')
fig.update_layout(bargap=0.1)
fig.show()

In [90]:
df_raw.columns

Index(['id', 'chol', 'stab.glu', 'hdl', 'ratio', 'glyhb', 'location', 'age',
       'gender', 'height', 'weight', 'frame', 'bp.1s', 'bp.1d', 'bp.2s',
       'bp.2d', 'waist', 'hip', 'time.ppn', 'height_meters', 'BMI',
       'hip_waist_ratio', 'diabetic'],
      dtype='object')

In [91]:
cols = ['id', 'bp.1s', 'bp.1d', 'height', 'location']

df_raw.drop(columns=cols)

,chol,stab.glu,hdl,ratio,glyhb,age,gender,weight,frame,bp.2s,bp.2d,waist,hip,time.ppn,height_meters,BMI,hip_waist_ratio,diabetic
0,203.0,27,56.0,3.6,4.310000,27,0,121.0,1,152.382979,92.524823,29.0,38.0,720.0,1.5748,48.790420,0.763158,0
1,165.0,42,24.0,6.9,4.440000,10,0,218.0,0,152.382979,92.524823,46.0,48.0,360.0,1.6256,82.495282,0.958333,0
2,228.0,37,37.0,6.2,4.640000,39,0,256.0,0,185.000000,92.000000,49.0,57.0,180.0,1.5494,106.638214,0.859649,0
3,78.0,38,12.0,6.5,4.630000,48,1,119.0,0,152.382979,92.524823,33.0,38.0,480.0,1.7018,41.089412,0.868421,0
4,249.0,35,28.0,8.9,7.720000,45,1,183.0,1,152.382979,92.524823,44.0,41.0,300.0,1.7272,61.343116,1.073171,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,296.0,113,46.0,6.4,16.110001,34,1,173.0,1,130.000000,94.000000,35.0,39.0,210.0,1.7526,56.322314,0.897436,1
399,284.0,34,54.0,5.3,4.390000,32,0,154.0,1,146.000000,102.000000,32.0,43.0,180.0,1.6002,60.141214,0.744186,0
400,194.0,105,38.0,5.1,13.630000,10,0,167.0,2,152.382979,92.524823,33.0,40.0,20.0,1.7526,54.368939,0.825000,1
401,199.0,21,52.0,3.8,4.490000,22,0,197.0,1,152.382979,92.524823,41.0,48.0,255.0,1.6002,76.933890,0.854167,0


In [92]:
df_raw.columns

Index(['id', 'chol', 'stab.glu', 'hdl', 'ratio', 'glyhb', 'location', 'age',
       'gender', 'height', 'weight', 'frame', 'bp.1s', 'bp.1d', 'bp.2s',
       'bp.2d', 'waist', 'hip', 'time.ppn', 'height_meters', 'BMI',
       'hip_waist_ratio', 'diabetic'],
      dtype='object')

In [93]:
df_raw.to_csv('diabetes_dropped_imputed_encoded.csv', index=None)

In [94]:
#saving the final dataset of prepared data for use in the modelling process.
df_import = pd.read_csv('../datasets/diabetes_dropped_imputed_encoded.csv')

In [95]:
df_import

,id,chol,stab.glu,hdl,ratio,glyhb,location,age,gender,height,...,bp.1d,bp.2s,bp.2d,waist,hip,time.ppn,height_meters,BMI,hip_waist_ratio,diabetic
0,0,203.0,27,56.0,3.6,4.310000,0,27,0,62.0,...,59.0,152.382979,92.524823,29.0,38.0,720.0,1.5748,48.790420,0.763158,0
1,1,165.0,42,24.0,6.9,4.440000,0,10,0,64.0,...,68.0,152.382979,92.524823,46.0,48.0,360.0,1.6256,82.495282,0.958333,0
2,2,228.0,37,37.0,6.2,4.640000,0,39,0,61.0,...,92.0,185.000000,92.000000,49.0,57.0,180.0,1.5494,106.638214,0.859649,0
3,3,78.0,38,12.0,6.5,4.630000,0,48,1,67.0,...,50.0,152.382979,92.524823,33.0,38.0,480.0,1.7018,41.089412,0.868421,0
4,4,249.0,35,28.0,8.9,7.720000,0,45,1,68.0,...,80.0,152.382979,92.524823,44.0,41.0,300.0,1.7272,61.343116,1.073171,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,398,296.0,113,46.0,6.4,16.110001,1,34,1,69.0,...,94.0,130.000000,94.000000,35.0,39.0,210.0,1.7526,56.322314,0.897436,1
399,399,284.0,34,54.0,5.3,4.390000,1,32,0,63.0,...,100.0,146.000000,102.000000,32.0,43.0,180.0,1.6002,60.141214,0.744186,0
400,400,194.0,105,38.0,5.1,13.630000,1,10,0,69.0,...,70.0,152.382979,92.524823,33.0,40.0,20.0,1.7526,54.368939,0.825000,1
401,401,199.0,21,52.0,3.8,4.490000,1,22,0,63.0,...,78.0,152.382979,92.524823,41.0,48.0,255.0,1.6002,76.933890,0.854167,0
